In [1]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

In [2]:
model = load_model('model-018.model/')
face_clfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
source = cv2.VideoCapture('.mp4')
labels_dict = {0: 'Mask', 1:'No Mask'}
color_dict = {0: (0,255,0), 1:(0,0,255)}

In [3]:
while(True):
    ret, img = source.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_clfr.detectMultiScale(gray, 1.3, 5)
    
    for x,y,w,h in faces:
        face_img = gray[y:y+h, x:x+w]
        resized = cv2.resize(face_img, (100,100))
        normalized = resized/255.0
        reshaped = np.reshape(normalized, (1,100,100,1))
        result = model.predict(reshaped)
        
        label = np.argmax(result, axis=1)[0]
        
        cv2.rectangle(img, (x,y), (x+w, y+h), color_dict[label], 2)
        cv2.rectangle(img, (x,y-40), (x+w, y), color_dict[label], -1)
        cv2.putText(img, labels_dict[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
        acc = round(np.max(result, axis=1)[0]*100, 2)
        cv2.putText(img, str(acc), (x+150, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
    cv2.imshow('LIVE', img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break

cv2.destroyAllWindows()
source.release()

TypeError: Argument 'delay' is required to be an integer